In [1]:
from ipywidgets import interact, interact_manual, widgets, Label
import numpy as np
import functools
from time import time, sleep
from os import system
import h5py

In [2]:
s_url='https://github.com/patrickhaddadteaching/ECB_rotrng/raw/main/data/all_data.zip'
a=system('wget %s'%s_url)
a=system('unzip all_data.zip')

In [3]:
hdf5_file = 'all_data.h5'
data_file = h5py.File(hdf5_file,'r')
ro_h=int(data_file['parameters/others_param'][0,1])
nb_osc=int(data_file['parameters/others_param'][0,2])
nb_test_per_corner=int(data_file['parameters/others_param'][0,3])
nb_bits=int(data_file['parameters/others_param'][0,4])
v_freq_sample_for_interact=data_file['parameters/v_freq_sample'][0,:]
v_jitter_for_interact=data_file['parameters/v_jitter'][0,:]

v_image_ros=[]
for i in range(nb_osc):
  v_image_ros.append((data_file['images/image_ro_%d'%i][0,:]).tobytes())
v_image_empty=(data_file['images/image_empty'][0,:]).tobytes()
v_image_xor=(data_file['images/image_xor'][0,:]).tobytes()
m_raw_bits_from_all_RO=data_file['raw_bits/m_raw_bits_from_all_RO'][:]
data_file.close()

In [4]:
raw_bytes=widgets.Textarea(value='RAW RANDOM STREAM',placeholder='',description='',disabled=True,layout=widgets.Layout(width='575px',height='%dpx'%(ro_h*nb_osc)))
#tests_results=widgets.Textarea(value='TEST RESULTS',placeholder='',description='',disabled=True,layout=widgets.Layout(width='800px',height='130px'))
tests_results=widgets.HTML(value='',placeholder='',description='',disabled=True,layout=widgets.Layout(width='800px',height='130px'))

v_s_freq_sample_for_interact=[]
v_s_order_of_mag=['Hz','KHz','MHz','GHz']
for i in range(v_freq_sample_for_interact.shape[0]):
  freq_sample_for_interact=v_freq_sample_for_interact[i]
  tmp_log10=int(np.log10(freq_sample_for_interact)/3)
  v_s_freq_sample_for_interact.append('%d%s'%(freq_sample_for_interact/(1000**(tmp_log10)),v_s_order_of_mag[tmp_log10]))


sample_freq_value_widgets=widgets.SelectionSlider(options=v_s_freq_sample_for_interact,description="F<sub>sample</sub>",disabled=False,continuous_update=False, orientation='horizontal',readout=True,readout_format='.1f')
std_dev_delay_value_widgets=widgets.SelectionSlider(options=v_jitter_for_interact,value=v_jitter_for_interact[1], description=u'\u03C3',disabled=False,continuous_update=False, orientation='horizontal',readout=True)
float_bar=widgets.FloatProgress(value=0.0,min=0.0, max=100.0,description='',bar_style='info',style={'bar_color': '#ff0000'},orientation='horizontal',layout=widgets.Layout(width='800px',height='50px'))

v_ena_button=[]
for i in range(nb_osc):
  v_ena_button.append(widgets.ToggleButton(value=True,description='',disabled=False,button_style='',tooltip='Description',icon='check',layout=widgets.Layout(width='20px',height='%dpx'%ro_h)))
v_box_ena_button=widgets.VBox(v_ena_button)

v_ro_images=[]
for i in range(nb_osc):
  v_ro_images.append(widgets.Image(value=v_image_ros[i], format='png'))
v_box_ro_images=widgets.VBox(v_ro_images)

def toggle_fig(obj,i):
  if obj['new']:  
    v_ro_images[i].value=v_image_ros[i]
  else:
    v_ro_images[i].value=v_image_empty

for i in range(nb_osc):
  v_ena_button[i].observe(functools.partial(toggle_fig, i=i),'value')

xor_images=widgets.Image(value=v_image_xor, format='png')

val_button=widgets.Button(description = 'GENERATE RANDOM VALUES',layout=widgets.Layout(width='172px'))
val_button.style.button_color = 'gray'


def run_fips_tests(v_bits):
  len_v_bits=v_bits.shape[0]
  nb_nibble=int((v_bits.shape[0])/4)
  v_nibble=np.zeros((nb_nibble,),dtype=np.uint8)
  for i in range(4):
    v_nibble+=np.array(v_bits[i:4*nb_nibble:4],dtype=np.uint8)*(2**i)

  T1=v_bits.sum()

  T2=0
  for j in range(16):
    fj=(v_nibble==j).sum()
    T2+=((fj-5000.0/16)**2)/(5000.0/16)

  nb_lambda=32
  m_T3=np.zeros((nb_lambda,2),dtype=np.float64)
  for lambda_i in range(nb_lambda):
    word_size_in_bits=lambda_i+3

    nb_words=int(len_v_bits-word_size_in_bits)
    v_words=np.zeros((nb_words,),dtype=np.uint64)
    for i in range(word_size_in_bits):
      v_words+=np.array(v_bits[i:i+nb_words],dtype=np.uint8)*(2**i)    

    patern_0=2**(word_size_in_bits-1)+1
    patern_1=(2**(word_size_in_bits)-2**(word_size_in_bits-1)-2)
    m_T3[lambda_i,0]=(v_words==patern_0).sum()
    m_T3[lambda_i,1]=(v_words==patern_1).sum()

  lambda_T4=34
  v_sum=np.zeros((len_v_bits-lambda_T4,),dtype=np.uint64)
  for i in range(lambda_T4):
    v_sum+=v_bits[i:len_v_bits-lambda_T4+i]

  T4_PS=((v_sum==0) | (v_sum==lambda_T4)).sum()

  v_T5=np.zeros((5000,),dtype=np.uint64)
  for tau in range(5000):
    v_T5[tau]=(v_bits[:5000]^v_bits[tau+1:tau+5000+1]).sum()
  return (T1,T2,[[m_T3[0,0],m_T3[0,1]],[m_T3[1,0],m_T3[1,1]],[m_T3[2,0],m_T3[2,1]],[m_T3[3,0],m_T3[3,1]],[m_T3[4,0],m_T3[4,1]],[m_T3[5:,0].sum(),m_T3[5:,1].sum()]],T4_PS,v_T5)


def generate_rnd(b):

  s_color_code_succeed='00FF00'
  s_color_code_failed='FF0000'

  tests_results.value=''
  raw_bytes.value=''
  chosen_freq_sample=sample_freq_value_widgets.value
  index_chosen_freq_sample=v_s_freq_sample_for_interact.index(chosen_freq_sample)
  chosen_jitter=std_dev_delay_value_widgets.value
  index_chosen_jitter=np.where(chosen_jitter==v_jitter_for_interact)[0][0]

  v_raw_bits=np.zeros((nb_bits,),dtype=np.bool)
  
  nb_osc_ena=0
  for i in range(nb_osc):
    if v_ena_button[i].value:
      tmp_rand=np.random.randint(0,nb_test_per_corner,dtype=np.uint32)
      v_raw_bits_from_one_osc=m_raw_bits_from_all_RO[i,index_chosen_jitter,index_chosen_freq_sample,tmp_rand,:]
      v_raw_bits^=v_raw_bits_from_one_osc
      nb_osc_ena+=1
  float_bar.value=0
  time_to_sleep=0.1*((1-((1.0*index_chosen_freq_sample)/v_freq_sample_for_interact.shape[0])))
  for i in range(100):
    sleep(time_to_sleep)
    float_bar.value=(i+1)
    
  (T1,T2,M_T3,T4_PS,v_T5)=run_fips_tests(v_raw_bits)

  #s_label_test='<p>%d Osc enabled\tSampling freq. %s</p>'%(nb_osc_ena,chosen_freq_sample)
  s_label_test=''
  if abs(T1-nb_bits/2)<346:
    s_color_code=s_color_code_succeed
  else:
    s_color_code=s_color_code_failed
  s_label_test='%s<table><tr><td style="color:#%s";>T1=%d</td><td>&nbsp&nbsp</td>'%(s_label_test,s_color_code,T1)

  if (T2<57.4)&(T2>1.03):
    s_color_code=s_color_code_succeed
  else:
    s_color_code=s_color_code_failed
  s_label_test='%s<td style="color:#%s";>T2=%.1f</td><td>&nbsp&nbsp</td>'%(s_label_test,s_color_code,T2)

  m_T3_bounds=[[2267,2733],[1079,1421],[502,748],[223,402],[90,223],[90,223]]

  """
  s_label_T3=''
  for b in range(2):
    for i in range(6):
      if (M_T3[i][b]<m_T3_bounds[i][1])&(M_T3[i][b]>m_T3_bounds[i][0]):
        s_color_code=s_color_code_succeed
      else:
        s_color_code=s_color_code_failed      
      s_label_T3='%s<p style="color:#%s";>T3(%d,%d)=%d</p>'%(s_label_T3,s_color_code,i+1,b,M_T3[i][b])
    s_label_T3='%s'%(s_label_T3)
  
  """

  s_label_T3='<table>'
  s_label_T3='%s<tr>'%(s_label_T3)
  s_label_T3="%s <td>&nbsp&nbsp\u03BB</td><td>&nbsp&nbsp</td>"%(s_label_T3)
  for i in range(6):
    s_label_T3="%s <td>&nbsp&nbsp%d</td><td>&nbsp&nbsp</td>"%(s_label_T3,i)
  s_label_T3='%s</tr>'%(s_label_T3)

  for b in range(2):
    s_label_T3='%s<tr>'%(s_label_T3)
    s_label_T3="%s <td>T<sub>3</sub>(%d,\u03BB)</td><td>&nbsp&nbsp</td>"%(s_label_T3,b)
    for i in range(6):
      if (M_T3[i][b]<m_T3_bounds[i][1])&(M_T3[i][b]>m_T3_bounds[i][0]):
        s_color_code=s_color_code_succeed
      else:
        s_color_code=s_color_code_failed       
      s_label_T3='%s <td style="color:#%s";>%d</td><td>&nbsp&nbsp</td>'%(s_label_T3,s_color_code,M_T3[i][b])
    s_label_T3='%s</tr>'%(s_label_T3)
  s_label_T3='%s</table>'%(s_label_T3)    

  if T4_PS==False:
    s_color_code=s_color_code_succeed
    s_result='SUCCEED'
  else:
    s_color_code=s_color_code_failed
    s_result='FAIL'
  s_label_test='%s<td style="color:#%s";>T4 %s</td><td>&nbsp&nbsp</td>'%(s_label_test,s_color_code,s_result)

  T5=v_T5.max()
  if (T5<2674)&(T5>2326):
    s_color_code=s_color_code_succeed
  else:
    s_color_code=s_color_code_failed

  s_label_test='%s<td style="color:#%s";>T5=%d</td></tr></table>'%(s_label_test,s_color_code,T5)

  s_label_test='%s<p></p>%s'%(s_label_test,s_label_T3)
  tests_results.value=s_label_test

  s_line=''
  for i in range(nb_bits):
    s_line='%s%d'%(s_line,v_raw_bits[i])
  raw_bytes.value=s_line

val_button.on_click(generate_rnd, remove=False)
v_box_top=widgets.VBox([widgets.HBox([v_box_ena_button,v_box_ro_images,xor_images,raw_bytes]),widgets.HBox([val_button,sample_freq_value_widgets,std_dev_delay_value_widgets]),float_bar,tests_results])
display(v_box_top)

In [5]:
nb_osc

14